In [4]:
from neo4j import GraphDatabase

class Neo4jGraphBuilder:
    def __init__(self, uri, user, password, database):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

    def close(self):
        self.driver.close()

    def create_node(self, concept):
        with self.driver.session() as session:
            session.run(
                "MERGE (c:Concept {name: $name})",
                name=concept
            )


    def create_relationship(self, node1, node2, relationship):
         # Remplacer les espaces par des underscores dans le nom de la relation
        sanitized_relationship = relationship.replace(" ", "_")
        with self.driver.session() as session:
            session.run(
            f"MATCH (a:Concept {{name: $node1}}), (b:Concept {{name: $node2}}) "
            f"MERGE (a)-[:{sanitized_relationship}]->(b)",
            node1=node1,
            node2=node2
            )
    def query_knowledge_graph(self, query):
        with self.driver.session() as session:
            result = session.run(query)
            return [record for record in result]

In [8]:

import pandas as pd

# Exemple de fonction pour construire le graphe
def build_graph(graph_file):
    # Remplacez par vos informations de connexion
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "abdou12345;"
    database = "graph2"

    # Instanciation de la classe de construction du graphe
    graph_builder = Neo4jGraphBuilder(uri, user, password, database)

    # Lire le fichier CSV avec pandas
    df = pd.read_csv(graph_file, sep="|")  # Assurez-vous que le fichier CSV utilise '|' comme séparateur

    # Parcourir chaque ligne du DataFrame pour créer des nœuds et des relations
    for index, row in df.iterrows():
        node1 = row['node_1']
        node2 = row['node_2']
        relationship = row['edge']

        # Création des nœuds
        graph_builder.create_node(node1)
        graph_builder.create_node(node2)

        # Créer une relation entre node1 et node2
        graph_builder.create_relationship(node1, node2, relationship)

    return graph_builder  



In [9]:
def search_concept(graph_builder, concept_name):
    query = f"MATCH (c:Concept {{name: '{concept_name}'}})-[r]->(n) RETURN c, r, n"
    results = graph_builder.query_knowledge_graph(query)
    
    return results

In [10]:

graph_file = 'data_output/graph2.csv'
graph_builder = build_graph(graph_file)


In [13]:
# Vérifiez si graph_builder n'est pas None avant de l'utiliser
if graph_builder is None:
    print("Erreur : graph_builder n'a pas été créé correctement.")
else:
    # Exemple d'utilisation : rechercher un concept
    concept_to_search = "risk"
    results = search_concept(graph_builder, concept_to_search)

    # Afficher les résultats
    for result in results:
        print(result)

    # Fermer la connexion après toutes les opérations
    graph_builder.close()

C:\Users\abdel\AppData\Local\Temp\ipykernel_13100\1893367139.py:29: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


<Record c=<Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:16' labels=frozenset({'Concept'}) properties={'name': 'risk'}> r=<Relationship element_id='5:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:1152921504606846992' nodes=(<Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:16' labels=frozenset({'Concept'}) properties={'name': 'risk'}>, <Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:64' labels=frozenset({'Concept'}) properties={'name': 'nition_project_risk'}>) type='use' properties={}> n=<Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:64' labels=frozenset({'Concept'}) properties={'name': 'nition_project_risk'}>>
<Record c=<Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:16' labels=frozenset({'Concept'}) properties={'name': 'risk'}> r=<Relationship element_id='5:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:1155173304420532240' nodes=(<Node element_id='4:2a9ff9cd-0ea3-4de0-86e3-3515726fc251:16' labels=frozenset({'Concept'}) properties={'name': 'risk'}>, <Node el

In [14]:
# Afficher les résultats de manière lisible
if results:
    print("Résultats de la recherche pour le concept :", concept_to_search)
    print("=" * 50)  # Ligne de séparation
    for record in results:
        # Récupérer les nœuds et la relation
        concept_node = record["c"]
        relationship = record["r"]
        related_node = record["n"]

        # Afficher les détails du nœud concept
        print(f"Concept: {concept_node['name']}")
        print(f"  - Relation: {relationship.type}")
        print(f"  - Lien vers: {related_node['name']}")
        print("=" * 50)  # Ligne de séparation
else:
    print("Aucun résultat trouvé pour le concept :", concept_to_search)


Résultats de la recherche pour le concept : risk
Concept: risk
  - Relation: use
  - Lien vers: nition_project_risk
Concept: risk
  - Relation: use
  - Lien vers: technique
Concept: risk
  - Relation: include
  - Lien vers: information
Concept: risk
  - Relation: include
  - Lien vers: project_risk_management
Concept: risk
  - Relation: include
  - Lien vers: condition
Concept: risk
  - Relation: include
  - Lien vers: stipulation
Concept: risk
  - Relation: perform
  - Lien vers: information
Concept: risk
  - Relation: project
  - Lien vers: team_member_opportunity
Concept: risk
  - Relation: recognize
  - Lien vers: result
Concept: risk
  - Relation: require
  - Lien vers: approval_response
Concept: risk
  - Relation: require
  - Lien vers: risk_management_process
Concept: risk
  - Relation: require
  - Lien vers: term_response
Concept: risk
  - Relation: require
  - Lien vers: diagram
Concept: risk
  - Relation: assess
  - Lien vers: priority
Concept: risk
  - Relation: assess
  - L

In [ ]:
def display_graph_info(self):
    # Récupérer les nœuds et les relations
    nodes_query = "MATCH (n) RETURN n LIMIT 5"
    relations_query = "MATCH ()-[r]->() RETURN r LIMIT 5"

    nodes = self.run_query(nodes_query)
    relations = self.run_query(relations_query)

    print("Nœuds dans le graphe :")
    for record in nodes:
        print(record)

    print("\nRelations dans le graphe :")
    for record in relations:
        print(record)
